In [26]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse

parser = argparse.ArgumentParser(description='This is a demonstration program')

# parser.add_argument('-batch_size', type=str, required=True, help='Please provide a batch_size')

# args = parser.parse_args()

# print(f'batch size: {args.batch_size}')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# batch_size = args.batch_size # to use the batch_size cmd arg -> python file_name.py -batch_size 32
batch_size = 64
block_size = 256
max_iters = 10000
learning_rate = 2e-5
eval_iters = 100
n_embd = 384
n_head = 4
# n_layer = 24
n_layer = 4
dropout = 0.2

print(device)

cuda


In [27]:
chars = ""
with open("vocab.txt", 'r', encoding='utf-8') as f:
        text = f.read()
        chars = sorted(list(set(text)))
        
vocab_size = len(chars)

In [28]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [29]:
# memory map for using small snippets of text from a single file of any size
def get_random_chunk(split):
    filename = "openwebtext_train.txt" if split == 'train' else "openwebtext_valid.txt"
    # filename = "output_train.txt" if split == 'train' else "output_valid.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data


def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [30]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out
    

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
    
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index
    
    # Add methods to make the model compatible with PEFT
    def prepare_inputs_for_generation(self, input_ids, **kwargs):
        """
        Prepare inputs for generation using this model.
        This is a required method for compatibility with PEFT.
        """
        return {"index": input_ids}
        
    def get_input_embeddings(self):
        """Return the token embedding layer (required for PEFT)"""
        return self.token_embedding_table
        
    def get_output_embeddings(self):
        """Return the LM head layer (required for PEFT)"""
        return self.lm_head

# model = GPTLanguageModel(vocab_size)
# print('loading model parameters...')
with open('model-01_60k.pkl', 'rb') as f:
# with open('model-01_60k_smallData.pkl', 'rb') as f:
    model = pickle.load(f)
print('loaded successfully!')
m = model.to(device)



loaded successfully!


In [22]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

@torch.no_grad()
def estimate_acc():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        accs = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, _ = model(X, Y)
            if len(logits.shape) == 3:
                B, T, C = logits.shape
                logits = logits.view(B*T, C)
            else:
                C = logits.shape[1]
            Y = Y.view(-1)  
            pred = logits.argmax(dim=-1)
            correct = (pred == Y).float().mean()
            accs[k] = correct.item()
        out[split] = accs.mean()
    model.train()
    return out

In [15]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
itr_list = []
train_acc_list = []
val_acc_list = []
train_loss_list = []
val_loss_list = []

for iter in range(max_iters):
    # print(iter)
    if iter % eval_iters == 0:
        losses = estimate_loss()
        accs = estimate_acc()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}, train acc: {accs['train']:.3f}, val acc: {accs['val']:.3f}")
        # Record metrics for plotting
        train_loss_list.append(losses['train'])
        val_loss_list.append(losses['val'])
        train_acc_list.append(accs['train'])
        val_acc_list.append(accs['val'])
        itr_list.append(iter)

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

with open('model-01_10k_largeData.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

step: 0, train loss: 8.613, val loss: 8.617, train acc: 0.000, val acc: 0.000
step: 100, train loss: 5.044, val loss: 5.046, train acc: 0.271, val acc: 0.267
step: 200, train loss: 3.995, val loss: 4.006, train acc: 0.280, val acc: 0.281
step: 300, train loss: 3.335, val loss: 3.334, train acc: 0.278, val acc: 0.280
step: 400, train loss: 2.984, val loss: 2.987, train acc: 0.281, val acc: 0.286
step: 500, train loss: 2.819, val loss: 2.822, train acc: 0.288, val acc: 0.289
step: 600, train loss: 2.713, val loss: 2.720, train acc: 0.287, val acc: 0.291
step: 700, train loss: 2.666, val loss: 2.653, train acc: 0.290, val acc: 0.291
step: 800, train loss: 2.599, val loss: 2.603, train acc: 0.290, val acc: 0.292
step: 900, train loss: 2.577, val loss: 2.579, train acc: 0.293, val acc: 0.298
step: 1000, train loss: 2.556, val loss: 2.546, train acc: 0.292, val acc: 0.295
step: 1100, train loss: 2.530, val loss: 2.525, train acc: 0.293, val acc: 0.293
step: 1200, train loss: 2.514, val loss:

In [ ]:
prompt = 'The Gambia also has'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=50)[0].tolist())
print(generated_chars)

TypeError: GPTLanguageModel.generate() got an unexpected keyword argument 'top_k'

In [ ]:
# Create a DataFrame with loss and accuracy metrics
import pandas as pd
df_metrics = pd.DataFrame({
    'iteration': itr_list,
    'train_loss': train_loss_list,
    'val_loss': val_loss_list,
    'train_acc': train_acc_list,
    'val_acc': val_acc_list
})

# Save to CSV
df_metrics.to_csv('training_metrics_largeData_30k.csv', index=False)

print("Metrics data saved to training_metrics_smallData_30k.csv")

Metrics data saved to training_metrics_smallData_30k.csv


In [ ]:
import re
import pandas as pd
df_metrics = pd.read_csv('/home/max/NLP-Final-Project/results/training_metrics_largeData_30k.csv')

# Visualize training metrics
import matplotlib.pyplot as plt


def to_scalar(x):
    # if it’s a real tensor, grab its .item()
    if hasattr(x, 'item'):
        return x.item()
    # if it’s a string like "tensor(1.2345)", pull out the number
    if isinstance(x, str):
        m = re.search(r"([-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?)", x)
        return float(m.group()) if m else float(x)
    # otherwise just cast
    return float(x)

# apply elementwise conversion
for col in ['train_loss','val_loss','train_acc','val_acc']:
    df_metrics[col] = df_metrics[col].apply(to_scalar)
# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17, 5))

# Plot loss
ax1.plot(df_metrics['iteration'], df_metrics['train_loss'], label='Training Loss')
ax1.plot(df_metrics['iteration'], df_metrics['val_loss'], label='Validation Loss')
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss (large dataset)')
ax1.legend()
ax1.grid(True)

# Plot accuracy
ax2.plot(df_metrics['iteration'], df_metrics['train_acc'], label='Training Accuracy')
ax2.plot(df_metrics['iteration'], df_metrics['val_acc'], label='Validation Accuracy')
ax2.set_xlabel('Iteration')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training and Validation Accuracy (large dataset)')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig('training_metrics_largeDataset_30k.png')
plt.show()

In [7]:

from datasets import load_dataset
import torch.nn.functional as F
import math, random, tqdm

# ── Load a small slice of OASST1 ──────────────────────────────────────────
ds = load_dataset("OpenAssistant/oasst1", split="train[:2%]")   
texts = ds["text"]

def encode_safe(txt: str):
    return [string_to_int[c] for c in txt if c in string_to_int]

# ── Inject LoRA into every Linear layer ───────────────────────────────────
class LoRALinear(nn.Module):
    """Additive LoRA wrapper for a frozen `nn.Linear`."""
    def __init__(self, base: nn.Linear, r: int = 8, alpha: int = 16):
        super().__init__()
        self.base = base                     
        for p in self.base.parameters():
            p.requires_grad = False

        self.r = r
        self.scaling = alpha / r
        self.lora_A = nn.Parameter(base.weight.new_zeros((r, base.in_features)))
        self.lora_B = nn.Parameter(base.weight.new_zeros((base.out_features, r)))
        nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
        nn.init.zeros_(self.lora_B)

    def forward(self, x):
        out = self.base(x)
        delta = F.linear(x, self.lora_B @ self.lora_A) * self.scaling
        return out + delta

def _patch(module):
    for name, child in module.named_children():
        if isinstance(child, nn.Linear):
            setattr(module, name, LoRALinear(child))  
        else:
            _patch(child)

_patch(model)  

# collect trainable params
lora_params = [p for p in model.parameters() if p.requires_grad]
print(f"LoRA parameters: {sum(p.numel() for p in lora_params):,}")

# ── Simple batch sampler ---------------------------------------------------
def get_batch(batch_size: int = 32):
    xs, ys = [], []
    while len(xs) < batch_size:
        ids = encode_safe(random.choice(texts))
        if len(ids) <= block_size:                 
            continue
        start = random.randint(0, len(ids) - block_size - 1)
        chunk = ids[start : start + block_size + 1]
        xs.append(chunk[:-1])
        ys.append(chunk[1:])
    x = torch.tensor(xs, dtype=torch.long, device=device)
    y = torch.tensor(ys, dtype=torch.long, device=device)
    return x, y

/home/max/anaconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LoRA parameters: 8,517,616


In [ ]:
# ── Train LoRA adapters ----------------------------------------------------
optimizer = torch.optim.Adam(lora_params, lr=1e-3)
steps = 10000                      

# Lists to track training metrics
lora_steps = []
lora_losses = []
lora_accuracies = []

# Function to compute accuracy
def compute_accuracy(logits, targets):
    # Reshape if needed
    if len(logits.shape) == 3:
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
    else:
        C = logits.shape[1]
        targets = targets.view(-1)
    # Get predictions and calculate accuracy
    preds = logits.argmax(dim=-1)
    correct = (preds == targets).float().mean()
    return correct.item()

model.train()
for step in tqdm.trange(steps):
    xb, yb = get_batch()
    logits, loss = model(xb, yb)      
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Record metrics every 100 steps
    if (step + 1) % 100 == 0:
        current_loss = loss.item()
        current_acc = compute_accuracy(logits, yb)
        print(f"step {step+1:>4}/{steps}  •  loss {current_loss:.4f}  •  acc {current_acc:.4f}")
        
        # Store metrics for plotting
        lora_steps.append(step + 1)
        lora_losses.append(current_loss)
        lora_accuracies.append(current_acc)

# ── Save just the adapters -------------------------------------------------
torch.save(
    {k: v.cpu() for k, v in model.state_dict().items() if "lora_" in k},
    "gptv1_lora_adapters.pt",
)
print("LoRA adapters saved to gptv1_lora_adapters.pt")


  1%|          | 103/10000 [00:07<11:10, 14.75it/s]

step  100/10000  •  loss 2.2334  •  acc 0.3997


  2%|▏         | 203/10000 [00:13<10:19, 15.81it/s]

step  200/10000  •  loss 2.0688  •  acc 0.3984


  3%|▎         | 303/10000 [00:20<10:27, 15.46it/s]

...

 99%|█████████▉| 9903/10000 [09:54<00:05, 16.63it/s] 

step 9900/10000  •  loss 1.5125  •  acc 0.5563


100%|██████████| 10000/10000 [10:00<00:00, 16.66it/s]

step 10000/10000  •  loss 1.5388  •  acc 0.5381
LoRA adapters saved to gptv1_lora_adapters.pt


NameError: name 'pd' is not defined

In [9]:
import pandas as pd
# Save training metrics to CSV
lora_metrics_df = pd.DataFrame({
    'iteration': lora_steps,
    'loss': lora_losses,
    'accuracy': lora_accuracies
})
lora_metrics_df.to_csv('lora_output/lora_training_metrics.csv', index=False)
print("LoRA training metrics saved to lora_output/lora_training_metrics.csv")

LoRA training metrics saved to lora_output/lora_training_metrics.csv


# Plot LoRA Training Metrics

Let's visualize the training loss and accuracy for the LoRA fine-tuning process.

In [10]:
# Plot LoRA training metrics
import matplotlib.pyplot as plt

# Create directory for results if it doesn't exist
import os
if not os.path.exists('lora_output'):
    os.makedirs('lora_output')

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot loss
ax1.plot(lora_steps, lora_losses, label='Training Loss', color='blue')
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Loss')
ax1.set_title('LoRA Training Loss')
ax1.grid(True)

# Plot accuracy
ax2.plot(lora_steps, lora_accuracies, label='Training Accuracy', color='green')
ax2.set_xlabel('Iteration')
ax2.set_ylabel('Accuracy')
ax2.set_title('LoRA Training Accuracy')
ax2.grid(True)

plt.tight_layout()
plt.savefig('lora_output/lora_training_metrics.png')
plt.show()

print("LoRA training metrics plot saved to lora_output/lora_training_metrics.png")

LoRA training metrics plot saved to lora_output/lora_training_metrics.png


In [ ]:
# Display the metrics dataframe
lora_metrics_df.head(10)

In [16]:
# ------------------ Quick interactive sampling ------------------
import torch, math, random, tqdm
from types import SimpleNamespace

# (Re)build / load the base model exactly as before -------------
# model = build_gpt_v1(...)   

# Re‑inject LoRA wrappers & load the adapters ------------------
# try:
#     next(m for m in model.modules() if isinstance(m, LoRALinear))
# except StopIteration:
#     _patch(model)                               

# lora_state = torch.load("gptv1_lora_adapters.pt", map_location="cpu")
# model.load_state_dict(lora_state, strict=False)  

model.to(device).eval()

# Helper: encode / decode at char level -----------------------
def encode_safe(txt):
    return [string_to_int[c] for c in txt if c in string_to_int]

def decode(ids):
    return "".join(int_to_string[i] for i in ids)

# Generate from a user prompt ---------------------------------
prompt = "the capital of United States"      
max_new = 200
temperature = 0.8
top_k = 40

ctx = torch.tensor([encode_safe(prompt)], dtype=torch.long, device=device)
model.eval()
with torch.no_grad():
    for _ in range(max_new):
        logits, _ = model(ctx)               
        logits = logits[:, -1, :] / temperature
        if top_k:
            v, _ = torch.topk(logits, top_k)
            logits[logits < v[:, [-1]]] = -float("Inf")
        probs = torch.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        ctx = torch.cat([ctx, next_id], dim=1)

print("-" * 40)
print(decode(ctx[0].tolist()))
print("-" * 40)


----------------------------------------
the capital of United States, Elgo D The S3, Automation. Aquanted encluye depending the S3 back, The S3 exampless of a like start feature in Austriction filef the discriminations idea is to could curle, it if you orders and even
----------------------------------------
